In [9]:
from peft import LoraConfig, TaskType
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer, BitsAndBytesConfig
# TRANSFORM=`python -c "import transformers;print('/'.join(transformers.__file__.split('/')[:-1])+'/models/llama/convert_llama_weights_to_hf.py')"`
# python ${TRANSFORM} --input_dir llama-2-13b --model_size 13B --output_dir llama-hf/13b

# MODEL_PATH = "../llama-hf/7b"
MODEL_PATH = "../llama-hf/13b"

In [11]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
tokenizer = LlamaTokenizer.from_pretrained(MODEL_PATH)
model = LlamaForCausalLM.from_pretrained(MODEL_PATH, quantization_config=bnb_config)

Loading checkpoint shards: 100%|██████████| 3/3 [00:25<00:00,  8.46s/it]


In [12]:
# prompt = "The tone of voice of the audio is"
prompt = """
Summarize this dialog:
A: Hi Tom, are you busy tomorrow’s afternoon?
B: I’m pretty sure I am. What’s up?
A: Can you go with me to the animal shelter?.
B: What do you want to do?
A: I want to get a puppy for my son.
B: That will make him so happy.
A: Yeah, we’ve discussed it many times. I think he’s ready now.
B: That’s good. Raising a dog is a tough issue. Like having a baby ;-) 
A: I'll get him one of those little dogs.
B: One that won't grow up too big;-)
A: And eat too much;-))
B: Do you know which one he would like?
A: Oh, yes, I took him there last Monday. He showed me one that he really liked.
B: I bet you had to drag him away.
A: He wanted to take it home right away ;-).
B: I wonder what he'll name it.
A: He said he’d name it after his dead hamster – Lemmy  - he's  a great Motorhead fan :-)))
---
Summary:
"""
inp = tokenizer(prompt, return_tensors="pt").to("cuda")
print(inp['input_ids'].device)

cuda:0


In [13]:
model.eval()
with torch.no_grad():
    out = model.generate(**inp, max_new_tokens=100)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [14]:
out.shape
tokenizer.decode(out[0])

"<s>\nSummarize this dialog:\nA: Hi Tom, are you busy tomorrow’s afternoon?\nB: I’m pretty sure I am. What’s up?\nA: Can you go with me to the animal shelter?.\nB: What do you want to do?\nA: I want to get a puppy for my son.\nB: That will make him so happy.\nA: Yeah, we’ve discussed it many times. I think he’s ready now.\nB: That’s good. Raising a dog is a tough issue. Like having a baby ;-) \nA: I'll get him one of those little dogs.\nB: One that won't grow up too big;-)\nA: And eat too much;-))\nB: Do you know which one he would like?\nA: Oh, yes, I took him there last Monday. He showed me one that he really liked.\nB: I bet you had to drag him away.\nA: He wanted to take it home right away ;-).\nB: I wonder what he'll name it.\nA: He said he’d name it after his dead hamster – Lemmy  - he's  a great Motorhead fan :-)))\n---\nSummary:\nA: Hi Tom, are you busy tomorrow’s afternoon?\nB: I’m pretty sure I am. What’s up?\nA: Can you go with me to the animal shelter?.\nB: What do you want

In [13]:
from llama_recipes.utils.dataset_utils import get_preprocessed_dataset
from llama_recipes.configs.datasets import samsum_dataset

train_dataset = get_preprocessed_dataset(tokenizer, samsum_dataset, 'train')

Map: 100%|██████████| 14732/14732 [00:02<00:00, 7325.69 examples/s]


In [14]:
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 6,553,600 || all params: 13,022,417,920 || trainable%: 0.05032552357220002


In [15]:
from transformers import TrainerCallback
from contextlib import nullcontext
enable_profiler = False
output_dir = "tmp/llama-output"

config = {
    'lora_config': peft_config,
    'learning_rate': 1e-4,
    'num_train_epochs': 1,
    'gradient_accumulation_steps': 2,
    'per_device_train_batch_size': 2,
    'gradient_checkpointing': False,
}

# Set up profiler
if enable_profiler:
    wait, warmup, active, repeat = 1, 1, 2, 1
    total_steps = (wait + warmup + active) * (1 + repeat)
    schedule =  torch.profiler.schedule(wait=wait, warmup=warmup, active=active, repeat=repeat)
    profiler = torch.profiler.profile(
        schedule=schedule,
        on_trace_ready=torch.profiler.tensorboard_trace_handler(f"{output_dir}/logs/tensorboard"),
        record_shapes=True,
        profile_memory=True,
        with_stack=True)
    
    class ProfilerCallback(TrainerCallback):
        def __init__(self, profiler):
            self.profiler = profiler
            
        def on_step_end(self, *args, **kwargs):
            self.profiler.step()

    profiler_callback = ProfilerCallback(profiler)
else:
    profiler = nullcontext()

In [39]:
from transformers import Trainer

In [16]:
from transformers import default_data_collator, Trainer, TrainingArguments

# Define training args
training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    bf16=False,  # Use BF16 if available
    # logging strategies
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=10,
    save_strategy="no",
    optim="adamw_torch_fused",
    max_steps=total_steps if enable_profiler else -1,
    **{k:v for k,v in config.items() if k != 'lora_config'}
)

with profiler:
    # Create Trainer instance
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        data_collator=default_data_collator,
        callbacks=[profiler_callback] if enable_profiler else [],
    )

    # Start training
    trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: loafboy72 (wien). Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/home/ubuntu/Jack/jenv/lib/python3.8/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,1.920900
20,1.795800
30,1.760300
40,1.660400
50,1.639900
60,1.643800
70,1.661700
80,1.650900
90,1.630200
100,1.636500


KeyboardInterrupt: 